In [3]:
# 2. Black–Scholes (B&S) Core Implementation

import numpy as np
from scipy.stats import norm
# import matplotlib.pyplot as plt

def black_scholes_call(S0, K, T, r, sigma, q=0):
    """
    Calculate European call option price using Black-Scholes formula
    
    Parameters:
    S0: Current stock price
    K: Strike price
    T: Time to expiration (years)
    r: Risk-free rate
    sigma: Volatility
    q: Dividend yield (default 0)
    """
    d1 = (np.log(S0/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    call_price = S0*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call_price

def black_scholes_put(S0, K, T, r, sigma, q=0):
    """
    Calculate European put option price using Black-Scholes formula
    """
    d1 = (np.log(S0/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S0*np.exp(-q*T)*norm.cdf(-d1)
    return put_price

# Example calculation
S0 = 100  # Current stock price
K = 100   # Strike price
T = 1     # 1 year to expiration
r = 0.05  # 5% risk-free rate
sigma = 0.2  # 20% volatility

call_price = black_scholes_call(S0, K, T, r, sigma)
put_price = black_scholes_put(S0, K, T, r, sigma)

print(f"Call Price: ${call_price:.2f}")
print(f"Put Price: ${put_price:.2f}")

Call Price: $10.45
Put Price: $5.57


In [1]:
# 4: OPM Implementation Framework for Breakpoints
class OPMEngine:
    def __init__(self):
        self.breakpoints = []
        self.share_classes = {}
        
    def add_breakpoint(self, value, description):
        """Add a breakpoint to the capital structure"""
        self.breakpoints.append({'value': value, 'description': description})
        
    def calculate_option_values(self, total_equity_value, volatility, risk_free_rate, time_to_exit):
        """Calculate option values for each slice of the capital structure"""
        # Sort breakpoints
        sorted_breakpoints = sorted(self.breakpoints, key=lambda x: x['value'])
        
        option_values = []
        
        for i, breakpoint in enumerate(sorted_breakpoints):
            # Calculate call option value for this slice
            strike = breakpoint['value']
            call_value = black_scholes_call(
                S0=total_equity_value,
                K=strike,
                T=time_to_exit,
                r=risk_free_rate,
                sigma=volatility
            )
            
            option_values.append({
                'breakpoint': breakpoint,
                'call_value': call_value
            })
        print(option_values)
        return option_values
    

# Example usage will be added in later sections

In [2]:
# 5. PWERM (Probability Weighted Expected Return Method) Implementation Framework
class PWERMEngine:
    def __init__(self):
        self.scenarios = []
        self.share_classes = {}
    
    def add_scenario(self, exit_value, probability, timing, discount_rate, description=""):
        """Add a scenario for PWERM analysis"""
        self.scenarios.append({
            'exit_value': exit_value,
            'probability': probability,
            'timing': timing,
            'discount_rate': discount_rate,
            'description': description
        })
    
    def calculate_scenario_values(self):
        """Calculate present value for each scenario"""
        scenario_results = []
        
        for scenario in self.scenarios:
            # Calculate present value
            pv_factor = (1 + scenario['discount_rate']) ** (-scenario['timing'])
            present_value = scenario['exit_value'] * pv_factor
            
            scenario_results.append({
                'scenario': scenario,
                'present_value': present_value,
                'weighted_value': present_value * scenario['probability']
            })
        
        return scenario_results
    
    def calculate_expected_value(self):
        """Calculate probability-weighted expected return"""
        scenario_results = self.calculate_scenario_values()
        return sum(result['weighted_value'] for result in scenario_results)

# Example PWERM setup
pwerm = PWERMEngine()
pwerm.add_scenario(exit_value=500_000_000, probability=0.3, timing=3, discount_rate=0.12, description="IPO Exit")
pwerm.add_scenario(exit_value=300_000_000, probability=0.5, timing=4, discount_rate=0.12, description="Strategic Sale")
pwerm.add_scenario(exit_value=100_000_000, probability=0.2, timing=5, discount_rate=0.12, description="Distressed Sale")

expected_value = pwerm.calculate_expected_value()
print(f"PWERM Expected Value: ${expected_value:,.0f}")

PWERM Expected Value: $213,443,286


In [4]:
# 7 & 8. CSE and DLOM Implementation Framework
import numpy as np

class CSEEngine:
    def __init__(self):
        self.securities = {}
        
    def add_security(self, name, shares, conversion_ratio=1.0, is_option=False, strike_price=None):
        """Add a security to the cap table"""
        self.securities[name] = {
            'shares': shares,
            'conversion_ratio': conversion_ratio,
            'is_option': is_option,
            'strike_price': strike_price
        }
    
    def calculate_common_equivalent_shares(self, current_stock_price=None):
        """Calculate common stock equivalent shares"""
        total_shares = 0
        
        for name, security in self.securities.items():
            if security['is_option'] and current_stock_price:
                # Treasury method for options
                if current_stock_price > security['strike_price']:
                    proceeds = security['shares'] * security['strike_price']
                    shares_purchased = proceeds / current_stock_price
                    net_shares = security['shares'] - shares_purchased
                    total_shares += net_shares * security['conversion_ratio']
            else:
                # Convert to common shares
                total_shares += security['shares'] * security['conversion_ratio']
        
        return total_shares

class DLOMCalculator:
    @staticmethod
    def protective_put_dlom(volatility, time_to_liquidity, risk_free_rate):
        """
        Calculate DLOM using Protective Put method
        Simplified version - real implementation would be more complex
        """
        from scipy.stats import norm
        
        # Black-Scholes put option formula (simplified for DLOM)
        d1 = (risk_free_rate + 0.5 * volatility**2) * time_to_liquidity / (volatility * np.sqrt(time_to_liquidity))
        d2 = d1 - volatility * np.sqrt(time_to_liquidity)
        
        put_value = np.exp(-risk_free_rate * time_to_liquidity) * norm.cdf(-d2) - norm.cdf(-d1)
        
        return put_value  # This is the DLOM as percentage
    
    @staticmethod
    def finnerty_dlom(revenue, ebitda_margin, volatility):
        """
        Simplified Finnerty model for DLOM
        Real implementation would include more factors
        """
        # Simplified formula based on company characteristics
        size_factor = max(0.05, min(0.30, 0.25 - np.log(revenue / 1_000_000) * 0.02))
        profitability_factor = max(0, 0.15 - ebitda_margin * 0.5)
        volatility_factor = min(0.20, volatility * 0.4)
        
        dlom = size_factor + profitability_factor + volatility_factor
        return min(dlom, 0.50)  # Cap at 50%

# Example DLOM calculation
dlom_calc = DLOMCalculator()

# Protective Put DLOM
protective_put_dlom = dlom_calc.protective_put_dlom(
    volatility=0.40,  # 40% volatility
    time_to_liquidity=2.0,  # 2 years to liquidity event
    risk_free_rate=0.05  # 5% risk-free rate
)

print(f"Protective Put DLOM: {protective_put_dlom:.1%}")

# Finnerty DLOM  
finnerty_dlom = dlom_calc.finnerty_dlom(
    revenue=50_000_000,  # $50M revenue
    ebitda_margin=0.15,  # 15% EBITDA margin
    volatility=0.30  # 30% volatility
)

print(f"Finnerty DLOM: {finnerty_dlom:.1%}")

Protective Put DLOM: 16.8%
Finnerty DLOM: 36.7%


In [5]:
# 9. Backsolve Implementation
from scipy.optimize import fsolve, brentq
import warnings
warnings.filterwarnings('ignore')

class BacksolveEngine:
    def __init__(self, omp_engine):
        self.omp_engine = omp_engine
        
    def backsolve_equity_value(self, target_class, target_pps, volatility, risk_free_rate, time_to_exit, initial_guess=100_000_000):
        """
        Backsolve for total equity value given a known price per share for a specific class
        
        Parameters:
        target_class: The security class with known transaction price
        target_pps: The known price per share for target_class
        volatility: Expected volatility
        risk_free_rate: Risk-free rate
        time_to_exit: Time to expected liquidity event
        initial_guess: Starting point for the solver
        """
        
        def objective_function(equity_value):
            """Function to minimize - difference between calculated and target PPS"""
            try:
                # Calculate OMP values for this equity value
                option_values = self.omp_engine.calculate_option_values(
                    total_equity_value=equity_value[0],
                    volatility=volatility,
                    risk_free_rate=risk_free_rate,
                    time_to_exit=time_to_exit
                )
                
                # Calculate PPS for target class (simplified logic)
                # In reality, this would involve complex waterfall calculations
                calculated_pps = self._calculate_class_pps(option_values, target_class, equity_value[0])
                
                return calculated_pps - target_pps
            except:
                return float('inf')  # Return large number if calculation fails
        
        # Use root finding to solve for equity value
        try:
            result = fsolve(objective_function, [initial_guess], xtol=1e-6)
            return result[0] if result[0] > 0 else None
        except:
            return None
    
    def _calculate_class_pps(self, option_values, target_class, total_equity_value):
        """
        Simplified PPS calculation for demonstration
        Real implementation would handle complex waterfall logic
        """
        # This is a simplified placeholder - real logic would be much more complex
        # involving liquidation preferences, participation rights, etc.
        
        if not option_values:
            return 0
            
        # Simple approximation for demonstration
        class_value = option_values[0]['call_value'] * 0.1  # Assume class gets 10% of option value
        shares_outstanding = 1_000_000  # Assume 1M shares for this class
        
        return class_value / shares_outstanding

# Example backsolve scenario
print("=== Backsolve Example ===")
print("Scenario: Series A raised $10M at $2.00/share")
print("Goal: Determine total equity value that supports this valuation")

# Setup OMP engine with some breakpoints
omp_engine = OPMEngine()
omp_engine.add_breakpoint(value=50_000_000, description="Debt Repayment")
omp_engine.add_breakpoint(value=100_000_000, description="Preferred Liquidation Preference")

# Create backsolve engine
backsolve = BacksolveEngine(omp_engine)

# Perform backsolve
solved_equity_value = backsolve.backsolve_equity_value(
    target_class="Series_A",
    target_pps=2.00,
    volatility=0.50,  # 50% volatility for early stage company
    risk_free_rate=0.05,
    time_to_exit=4.0,  # 4 years to exit
    initial_guess=150_000_000
)

if solved_equity_value:
    print(f"Backsolve Result: Total Equity Value = ${solved_equity_value:,.0f}")
    print(f"This implies the company was valued at ${solved_equity_value:,.0f} total equity value")
else:
    print("Backsolve failed - no solution found")

=== Backsolve Example ===
Scenario: Series A raised $10M at $2.00/share
Goal: Determine total equity value that supports this valuation
Backsolve Result: Total Equity Value = $150,000,000
This implies the company was valued at $150,000,000 total equity value


In [9]:
# 10. Mapping Excel OPM → Python OPM Complete OPM Engine Integration
import pandas as pd
from datetime import datetime
import json
import numpy as np
from scipy.stats import norm

def black_scholes_call(S0, K, T, r, sigma, q=0):
    """Black-Scholes call option pricing formula"""
    d1 = (np.log(S0/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    call_value = S0*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call_value

class ComprehensiveOPMEngine:
    """
    Full-featured OPM engine that mirrors Excel functionality
    """
    
    def __init__(self):
        self.cap_table = pd.DataFrame()
        self.breakpoints = []
        self.parameters = {}
        self.results = {}
        
    def load_cap_table(self, cap_table_data):
        """Load capitalization table from various formats"""
        if isinstance(cap_table_data, dict):
            self.cap_table = pd.DataFrame(cap_table_data)
        elif isinstance(cap_table_data, pd.DataFrame):
            self.cap_table = cap_table_data.copy()
        else:
            raise ValueError("Cap table must be dict or DataFrame")
    
    def set_parameters(self, equity_value, volatility, risk_free_rate, time_to_exit, dividend_yield=0):
        """Set OPM parameters"""
        self.parameters = {
            'equity_value': equity_value,
            'volatility': volatility,
            'risk_free_rate': risk_free_rate,
            'time_to_exit': time_to_exit,
            'dividend_yield': dividend_yield,
            'valuation_date': datetime.now()
        }
    
    def calculate_breakpoints_from_cap_table(self):
        """Automatically calculate breakpoints from cap table"""
        breakpoints = []
        
        # Add debt repayment breakpoints
        if 'debt_amount' in self.cap_table.columns:
            total_debt = self.cap_table['debt_amount'].sum()
            if total_debt > 0:
                breakpoints.append({
                    'value': total_debt,
                    'description': 'Total Debt Repayment',
                    'type': 'debt'
                })
        
        # Add liquidation preference breakpoints
        if 'liquidation_preference' in self.cap_table.columns:
            for idx, row in self.cap_table.iterrows():
                if pd.notna(row['liquidation_preference']) and row['liquidation_preference'] > 0:
                    breakpoints.append({
                        'value': row['liquidation_preference'],
                        'description': f"{row['security_name']} Liquidation Preference",
                        'type': 'liquidation_preference',
                        'security': row['security_name']
                    })
        
        # Add participation caps
        if 'participation_cap' in self.cap_table.columns:
            for idx, row in self.cap_table.iterrows():
                if pd.notna(row['participation_cap']) and row['participation_cap'] > 0:
                    breakpoints.append({
                        'value': row['participation_cap'],
                        'description': f"{row['security_name']} Participation Cap",
                        'type': 'participation_cap',
                        'security': row['security_name']
                    })
        
        # Sort breakpoints by value
        self.breakpoints = sorted(breakpoints, key=lambda x: x['value'])
        return self.breakpoints
    
    def run_full_opm_analysis(self):
        """Run complete OPM analysis"""
        if not self.parameters:
            raise ValueError("Parameters not set. Call set_parameters() first.")
        
        # Calculate breakpoints if not already done
        if not self.breakpoints:
            self.calculate_breakpoints_from_cap_table()
        
        # Calculate option values for each breakpoint
        option_values = []
        prev_value = 0
        
        for breakpoint in self.breakpoints:
            # Call option value at this strike
            call_value = black_scholes_call(
                S0=self.parameters['equity_value'],
                K=breakpoint['value'],
                T=self.parameters['time_to_exit'],
                r=self.parameters['risk_free_rate'],
                sigma=self.parameters['volatility'],
                q=self.parameters['dividend_yield']
            )
            
            # Incremental option value for this tranche
            incremental_value = call_value - prev_value
            prev_value = call_value
            
            option_values.append({
                'breakpoint': breakpoint,
                'call_value': call_value,
                'incremental_value': incremental_value
            })
        
        # Allocate values to security classes
        self.results = self._allocate_to_securities(option_values)
        
        return self.results
    
    def _allocate_to_securities(self, option_values):
        """Allocate option values to individual security classes"""
        # Simplified allocation logic - real implementation would be much more complex
        results = {}
        
        for idx, row in self.cap_table.iterrows():
            security_name = row['security_name']
            shares_outstanding = row.get('shares_outstanding', 0)
            
            # Simple allocation based on seniority and rights
            # Real logic would handle complex waterfall calculations
            allocated_value = 0
            
            if shares_outstanding > 0:
                # Placeholder allocation logic
                total_option_value = sum(ov['incremental_value'] for ov in option_values)
                ownership_pct = row.get('ownership_percentage', 0) / 100
                allocated_value = total_option_value * ownership_pct
                
                pps = allocated_value / shares_outstanding
            else:
                pps = 0
            
            results[security_name] = {
                'total_value': allocated_value,
                'shares_outstanding': shares_outstanding,
                'price_per_share': pps,
                'security_details': dict(row)
            }
        
        return results
    
    def export_results(self, format='json'):
        """Export results in various formats"""
        if format == 'json':
            return json.dumps(self.results, indent=2, default=str)
        elif format == 'dataframe':
            return pd.DataFrame(self.results).T
        else:
            raise ValueError("Format must be 'json' or 'dataframe'")

# Example usage with sample cap table
sample_cap_table = {
    'security_name': ['Common Stock', 'Series A Preferred', 'Series B Preferred', 'Employee Options'],
    'shares_outstanding': [5_000_000, 2_000_000, 1_500_000, 500_000],
    'liquidation_preference': [0, 10_000_000, 20_000_000, 0],
    'participation_cap': [0, 0, 40_000_000, 0],
    'ownership_percentage': [40, 30, 20, 10]
}

# Initialize and run OPM
omp_engine = ComprehensiveOPMEngine()

# Load cap table
omp_engine.load_cap_table(sample_cap_table)

# Set parameters
omp_engine.set_parameters(
    equity_value=100_000_000,
    volatility=0.45,
    risk_free_rate=0.05,
    time_to_exit=3.0
)

# Run analysis
results = omp_engine.run_full_opm_analysis()

# Display results
print("=== OPM Analysis Results ===")
for security, data in results.items():
    print(f"{security}:")
    print(f"  Total Value: ${data['total_value']:,.0f}")
    print(f"  Price Per Share: ${data['price_per_share']:.2f}")
    print(f"  Shares Outstanding: {data['shares_outstanding']:,}")
    print()

=== OPM Analysis Results ===
Common Stock:
  Total Value: $26,910,767
  Price Per Share: $5.38
  Shares Outstanding: 5,000,000

Series A Preferred:
  Total Value: $20,183,075
  Price Per Share: $10.09
  Shares Outstanding: 2,000,000

Series B Preferred:
  Total Value: $13,455,383
  Price Per Share: $8.97
  Shares Outstanding: 1,500,000

Employee Options:
  Total Value: $6,727,692
  Price Per Share: $13.46
  Shares Outstanding: 500,000

